In [2]:
import datetime as dt
import pandas as pd
import joblib

In [3]:
df_train = pd.read_csv('training.csv', index_col='Date') # 2003 - 2013
df_val = pd.read_csv('validation.csv', index_col='Date') # 2017/08/3
df_pred = pd.read_csv('prediction.csv', index_col='Date') # 2024/07/23

MODEL_PATH = "model.pkl"
model = joblib.load(MODEL_PATH)

In [4]:
display(df_train.columns)

#display(df_val)
df_pred.drop(columns=['Quarterly Revenue Growth  (yoy)', 'Operating Cash Flow  (ttm)'], inplace=True)

df_pred.rename(columns={
    'PEG Ratio (5yr expected)': 'PEG Ratio',
    'Operating Margin  (ttm)': 'Operating Margin',
    'Return on Assets  (ttm)': 'Return on Assets',
    'Return on Equity  (ttm)': 'Return on Equity', 
    'Revenue  (ttm)': 'Revenue',
    'Revenue Per Share  (ttm)': 'Revenue Per Share',
    'Net Income Avi to Common  (ttm)': 'Net Income Avl to Common',
    'Diluted EPS  (ttm)': 'Diluted EPS',
    'Total Cash  (mrq)': 'Total Cash',
    'Total Cash Per Share  (mrq)': 'Total Cash Per Share',
    'Total Debt  (mrq)': 'Total Debt',
    'Total Debt/Equity  (mrq)': 'Total Debt/Equity',
    'Current Ratio  (mrq)': 'Current Ratio',
    'Book Value Per Share  (mrq)': 'Book Value Per Share',
    'Beta (5Y Monthly)': 'Beta',
    '50-Day Moving Average3': '50-Day Moving Average',
    '200-Day Moving Average3': '200-Day Moving Average',
    'Avg Vol (3 month)3': 'Avg Vol (3 month)',
    'Shares Outstanding5': 'Shares Outstanding'

}, inplace=True)

display(df_pred.columns)
# drop Gross Profit from 1

# drop 'Quarterly Revenue Growth  (yoy) from 2' 'Operating Cash Flow  (ttm)'

Index(['Ticker', 'Price', 'stock_p_change', 'SP500', 'SP500_p_change',
       'Market Cap', 'Enterprise Value', 'Trailing P/E', 'Forward P/E',
       'PEG Ratio', 'Price/Sales', 'Price/Book', 'Enterprise Value/Revenue',
       'Enterprise Value/EBITDA', 'Profit Margin', 'Operating Margin',
       'Return on Assets', 'Return on Equity', 'Revenue', 'Revenue Per Share',
       'EBITDA', 'Net Income Avl to Common', 'Diluted EPS', 'Total Cash',
       'Total Cash Per Share', 'Total Debt', 'Total Debt/Equity',
       'Current Ratio', 'Book Value Per Share', 'Beta',
       '50-Day Moving Average', '200-Day Moving Average', 'Avg Vol (3 month)',
       'Shares Outstanding'],
      dtype='object')

Index(['Ticker', 'Market Cap', 'Enterprise Value', 'Trailing P/E',
       'Forward P/E', 'PEG Ratio', 'Price/Sales', 'Price/Book',
       'Enterprise Value/Revenue', 'Enterprise Value/EBITDA', 'Profit Margin',
       'Operating Margin', 'Return on Assets', 'Return on Equity', 'Revenue',
       'Revenue Per Share', 'EBITDA', 'Net Income Avl to Common',
       'Diluted EPS', 'Total Cash', 'Total Cash Per Share', 'Total Debt',
       'Total Debt/Equity', 'Current Ratio', 'Book Value Per Share', 'Beta',
       '50-Day Moving Average', '200-Day Moving Average', 'Avg Vol (3 month)',
       'Shares Outstanding', 'Predictions'],
      dtype='object')

In [6]:
df1_columns = [col for i, col in enumerate(df_train.columns) if i not in [1, 2, 3, 4]]
df2_columns = [col for i, col in enumerate(df_pred.columns) if i not in []]

for col1, col2 in zip(df1_columns, df2_columns):
    print(f"df1: {col1} df2: {col2}")

df1: Ticker df2: Ticker
df1: Market Cap df2: Market Cap
df1: Enterprise Value df2: Enterprise Value
df1: Trailing P/E df2: Trailing P/E
df1: Forward P/E df2: Forward P/E
df1: PEG Ratio df2: PEG Ratio
df1: Price/Sales df2: Price/Sales
df1: Price/Book df2: Price/Book
df1: Enterprise Value/Revenue df2: Enterprise Value/Revenue
df1: Enterprise Value/EBITDA df2: Enterprise Value/EBITDA
df1: Profit Margin df2: Profit Margin
df1: Operating Margin df2: Operating Margin
df1: Return on Assets df2: Return on Assets
df1: Return on Equity df2: Return on Equity
df1: Revenue df2: Revenue
df1: Revenue Per Share df2: Revenue Per Share
df1: EBITDA df2: EBITDA
df1: Net Income Avl to Common df2: Net Income Avl to Common
df1: Diluted EPS df2: Diluted EPS
df1: Total Cash df2: Total Cash
df1: Total Cash Per Share df2: Total Cash Per Share
df1: Total Debt df2: Total Debt
df1: Total Debt/Equity df2: Total Debt/Equity
df1: Current Ratio df2: Current Ratio
df1: Book Value Per Share df2: Book Value Per Share
df1:

In [9]:
X = df_pred.iloc[:, 1:]

In [10]:
predictions = model.predict(X)

In [11]:
preds = pd.Series(predictions, name='Predictions')
#X_original = pd.DataFrame(scaler.inverse_transform(X), columns=X.columns)
merged = pd.concat([df_pred.reset_index(drop=True), preds.reset_index(drop=True)], axis=1)

stock_symbols = merged[merged['Predictions'] == True]['Ticker'].to_list()

In [12]:
start = dt.datetime(2024, 7, 23)
end = dt.datetime.now()
results = []
average = 0
import yfinance as yf

for stock in stock_symbols:
    df = yf.download(stock, start=start, end=end)['Adj Close']
    if len(df) > 0:
        pct_change = df[-1] / df[0]
        results.append((stock, pct_change))
        average += pct_change

for stock, pct_change in results:
    print(f"Stock: {stock}, Percentage Change: {(pct_change-1):.2%}")
print(f"{(average/len(results) - 1) * 100} %")

[*********************100%%**********************]  1 of 1 completed
C:\Users\Bogdan\AppData\Local\Temp\ipykernel_22652\458884508.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pct_change = df[-1] / df[0]
[*********************100%%**********************]  1 of 1 completed
C:\Users\Bogdan\AppData\Local\Temp\ipykernel_22652\458884508.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pct_change = df[-1] / df[0]
[*********************100%%**********************]  1 of 1 completed
C:\Users\Bogdan\AppData\Local\Temp\ipykernel_22652\458884508.py:10: FutureWarning: Series.__getitem__ treating keys as positions is d

Stock: APD, Percentage Change: 7.76%
Stock: COR, Percentage Change: 6.24%
Stock: CPAY, Percentage Change: 2.96%
Stock: CSGP, Percentage Change: 2.87%
Stock: CRWD, Percentage Change: 0.99%
Stock: DHI, Percentage Change: 8.81%
Stock: ETN, Percentage Change: -6.50%
Stock: FFIV, Percentage Change: 14.51%
Stock: FSLR, Percentage Change: 3.61%
Stock: IVZ, Percentage Change: 3.64%
Stock: LHX, Percentage Change: -3.89%
Stock: LRCX, Percentage Change: -11.70%
Stock: LEN, Percentage Change: 7.16%
Stock: LULU, Percentage Change: -4.56%
Stock: RL, Percentage Change: 4.53%
Stock: RMD, Percentage Change: 8.42%
Stock: SPGI, Percentage Change: 1.02%
Stock: SNA, Percentage Change: 4.00%
Stock: LUV, Percentage Change: 3.68%
Stock: TEL, Percentage Change: 1.55%
Stock: ULTA, Percentage Change: -2.73%
Stock: VRTX, Percentage Change: -1.85%
Stock: GWW, Percentage Change: 3.47%
Stock: WBA, Percentage Change: -6.09%
Stock: WTW, Percentage Change: 5.65%
2.141768606962846 %



C:\Users\Bogdan\AppData\Local\Temp\ipykernel_22652\458884508.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pct_change = df[-1] / df[0]


In [14]:
merged.to_csv('final.csv')